In [ ]:
import json
import numpy as np
import matplotlib.pyplot as plt
from utils.udp_util import extract_udp_prague_to_dataframe
from utils.plotter import  plot_siftr_graph

In [ ]:
# File paths


duration = 120  # Duration in seconds
filename="udp_net_2025-08-07-14-21-30_baseline"




# filename="udp_net_2025-08-07-14-15-15_baseline"

# filename="udp_net_2025-08-07-14-18-07_best"

filename="udp_net_2025-08-07-14-51-01_best"
duration = 1800


file_udp = f'../data/{filename}/udp_prague_receiver_iperf3_d{duration}.json'
file_cubic = f'../data/{filename}/iperf3_client_cubic_iperf3_d{duration}.json'

utf = extract_udp_prague_to_dataframe(file_udp)

from utils.util import create_directory_if_not_exists
import os
graph_directory = os.path.join("./", "Graphs")
create_directory_if_not_exists(graph_directory)


In [ ]:
import json
import numpy as np
import pandas as pd

def extract_data_df(filename):
    with open(filename) as f:
        data = json.load(f)

    records = []

    for interval in data['intervals']:
        record = {
            'time': interval['sum']['start'],
            'bitrate_mbps': interval['sum']['bits_per_second'] / 1_000_000
        }

        stream = interval['streams'][0]
        if 'rtt' in stream:
            record['rtt_time'] = stream['start']
            record['rtt_ms'] = stream['rtt'] / 1000.0  # µs to ms
        else:
            record['rtt_time'] = None
            record['rtt_ms'] = None

        records.append(record)

    df = pd.DataFrame(records)
    return df


In [ ]:
import json
import pandas as pd

def extract_data_df(filename):
    with open(filename) as f:
        data = json.load(f)

    records = []

    for interval in data['intervals']:
        record = {
            'time': interval['sum']['start'],
            'bitrate_mbps': interval['sum']['bits_per_second'] / 1_000_000
        }

        stream = interval['streams'][0]

        # RTT
        if 'rtt' in stream:
            record['rtt_time'] = stream['start']
            record['rtt_ms'] = stream['rtt'] / 1000.0  # µs to ms
        else:
            record['rtt_time'] = None
            record['rtt_ms'] = None

        # Retransmits
        record['retransmits'] = stream.get('retransmits', None)

        # Send congestion window (bytes to KB)
        if 'snd_cwnd' in stream:
            record['snd_cwnd_kb'] = stream['snd_cwnd'] / 1024.0
        else:
            record['snd_cwnd_kb'] = None

        records.append(record)

    df = pd.DataFrame(records)
    return df


In [ ]:



# # Function to extract data from iperf3 JSON
# def extract_data(filename):
#     with open(filename) as f:
#         data = json.load(f)

#     times = []
#     bitrates = []
#     rtt_times = []
#     rtts = []

#     for interval in data['intervals']:
#         times.append(interval['sum']['start'])
#         bitrates.append(interval['sum']['bits_per_second'] / 1_000_000)  # Convert to Mbit/s

#         stream = interval['streams'][0]
#         if 'rtt' in stream:
#             rtt_times.append(stream['start'])
#             rtts.append(stream['rtt'] / 1000.0)  # µs to ms

#     return np.array(times), np.array(bitrates), np.array(rtt_times), np.array(rtts)





# import os
# os.path.exists(file_cubic)

# # Extract data
# t_cubic, b_cubic, rt_cubic, rtt_cubic = extract_data(file_cubic)

In [ ]:
tfcubic = extract_data_df(file_cubic)

In [ ]:
utf.rename(columns={'RTT_ms': 'SmoothedRTT'}, inplace=True)
tfcubic.rename(columns={'rtt_ms': 'SmoothedRTT'}, inplace=True)


utf.rename(columns={'time': 'Time'}, inplace=True)
tfcubic.rename(columns={'time': 'Time'}, inplace=True)


utf.rename(columns={'Rcvd_Mbps': 'thrpt'}, inplace=True)
tfcubic.rename(columns={'bitrate_mbps': 'thrpt'}, inplace=True)


utf.rename(columns={'Lost_Lost': 'Lost_Packets'}, inplace=True)
tfcubic.rename(columns={'retransmits': 'Lost_Packets'}, inplace=True)

In [ ]:
tfcubic.describe()

In [ ]:
utf.describe()

In [ ]:
utf

In [ ]:
tfcubic

In [ ]:
paths = {
    "Cubic": tfcubic,
    "L4S": utf,
}

utf.to_csv("./graph_data/udp_prague_rtt.csv")
tfcubic.to_csv("./graph_data/cubic_rtt.csv")


In [ ]:
import pandas as pd

# Assuming you already have tfcubic loaded
q1 = tfcubic['SmoothedRTT'].quantile(0.25)
q3 = tfcubic['SmoothedRTT'].quantile(0.75)
iqr = q3 - q1

# Define bounds for non-outliers
lower_bound = q1 - 1.5 * iqr
upper_bound = q3 + 1.5 * iqr

# Filter out outliers
tfcubic_cleaned = tfcubic[(tfcubic['SmoothedRTT'] >= lower_bound) & (tfcubic['SmoothedRTT'] <= upper_bound)]


import pandas as pd

# Assuming you already have utf loaded
q1 = utf['SmoothedRTT'].quantile(0.25)
q3 = utf['SmoothedRTT'].quantile(0.75)
iqr = q3 - q1

# Define bounds for non-outliers
lower_bound = q1 - 1.5 * iqr
upper_bound = q3 + 1.5 * iqr

# Filter out outliers
utf_cleaned = utf[(utf['SmoothedRTT'] >= lower_bound) & (utf['SmoothedRTT'] <= upper_bound)]


In [ ]:
    # Define paths
paths = {
    "Cubic": tfcubic_cleaned,
    "L4S": utf_cleaned,
}


# paths = {
#     "Cubic": tfcubic_filtered_rtt,
#     "L4S": utf_filtered_rtt,
#     "Baseline Propagation Delay": baseline_propagation_delay_df
# }


scenario = str("RTT_UDPPraguevsCubic")

plot_siftr_graph(paths=paths,
                ycolumn="SmoothedRTT",
                title=f"{scenario} SmoothedRTT",
                xlabel="Time (s)",
                ylabel="SmoothedRTT (ms)",
                filename=f'{scenario}',
                graph_directory=graph_directory,
)

In [ ]:
    # Define paths
paths = {
    "Cubic": tfcubic,
    "L4S": utf,
}


# paths = {
#     "Cubic": tfcubic_filtered_rtt,
#     "L4S": utf_filtered_rtt,
#     "Baseline Propagation Delay": baseline_propagation_delay_df
# }


scenario = str("RTT_UDPPraguevsCubic")

plot_siftr_graph(paths=paths,
                ycolumn="SmoothedRTT",
                title=f"{scenario} SmoothedRTT",
                xlabel="Time (s)",
                ylabel="SmoothedRTT (ms)",
                filename=f'{scenario}',
                graph_directory=graph_directory,
)

In [ ]:
    # Define paths
paths = {
    "Cubic": tfcubic,
    "L4S": utf,

}

scenario = str("Thrpt_UDPPraguevsCubic")

plot_siftr_graph(paths=paths,
                ycolumn="thrpt",
                title=f"{scenario} Throuhghput",
                xlabel="Time (s)",
                ylabel="Throuhghput (mbps)",
                filename=f'{scenario}',
                graph_directory=graph_directory,
)

In [ ]:
tfcubic[['Time', 'Lost_Packets']].to_csv("./graph_data/cubic_loss.csv", index=False)
utf[['Time', 'Lost_Packets']].to_csv("./graph_data/udp_prague_loss.csv", index=False)


In [ ]:
    # Define paths
paths = {
    "Cubic": tfcubic,
    "L4S": utf,
    # "Bandwidth": baseline_throuhgput_df
}



scenario = str("PktLoss_UDPPraguevsCubic")

plot_siftr_graph(paths=paths,
                ycolumn="Lost_Packets",
                title=f"{scenario} Packet Loss",
                xlabel="Time (s)",
                ylabel="Packet Loss",
                filename=f'{scenario}',
                graph_directory=graph_directory,
)